# Тестіруєм ДІЯ підпис зі сторони OnBoarding і другого PravexOnLine 

## Параметри підключення до бази даних 
Потрібно внести  oracle login  та oracle  пароль для підключення. Можливо підправити реквізити підключеня до БД

In [ ]:
import oracledb

import getpass
userpwd = getpass.getpass("Enter password: ")
username = "ORACLE Username"
#userpwd = ""
host="10.11.4.16"
port="1722"
service_name="TORCL"
dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'

#print(dsn)

In [28]:
import requests
import  unittest
import sys
import logging
import oracledb
import pytest
import json
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

class TestNotebook(unittest.TestCase):
   
    shared_data={}
    
    def setUp(self):
        log=logging.getLogger('setUp')
        log.info("=======Підключаюся до бази даних===============")
        self.connection = oracledb.connect(dsn)

    def tearDown(self):
        log=logging.getLogger('tearDown')
        log.info("=======Відключаюся від бази даних ===============")
        self.connection.close()

  
    def test_sign_docs(self):
        """================ Пошук заявки на підпис документів через ДІЯ ================"""
        log=logging.getLogger('test_sign_docs')
        log.info( f"Опис тесту: {self.shortDescription()}" )

        log.info(f"Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ")
        with self.connection.cursor() as cursor:
            sql = """ select  x.idprocess , z.zosys_id, t.fid, count(*) AS doccnt from qpf.QPF$FXCUST_ZF_EDOC t
                            inner join qpf.QPF$FXCUST_ZF z on z.fid=t.fid 
                            inner join qpf.QPF$FXCUST_ZF$EXTID x on x.zosys_id=z.zosys_id
                      where t.ecp_0 is null  
                            and t.edoc_hesh is not null
                            and t.edoc_type IN ('AML_OPROS','CASA_CABBAGE_CONTRCOMPLEX','NOTE_FGVFO','CARDACC_FORM', 'CRS_I', 'EDR_FORM', 'CASA_CABBAGE_FOPCOMPLEX', 'CURRACC_FORM')
                      group by x.idprocess , z.zosys_id, t.fid
                      order by t.fid desc"""
            cursor.execute(sql)
            log.info("Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)")
            columns = [col[0] for col in cursor.description]
            cursor.rowfactory = lambda *args: dict(zip(columns, args))
            dbdata = cursor.fetchone()
            log.info(f"Вибрано row: {dbdata}")
        log.info("Формую json структуру для http запиту")
        req={
            "Header": {
                        "SystemId": "XZ",
                        "CallID": "XZ-111-5",
                        "DtRequest": "2024-07-30T12:05:49.440Z"
            },
            "Request": {
                 "src": "SEBFL.ONBOARDING",
                 "idprocess": dbdata["IDPROCESS"],
                 "returnLink": "https://domain/returnlink"   
            }
        } 
        TestNotebook.shared_data["src"]="SEBFL.ONBOARDING"
        TestNotebook.shared_data["idprocess"]= dbdata["IDPROCESS"]
        req_body=json.dumps( req )
        req_url="http://prx-int-ace1:7816/apptoapp_srvc/v1/diiasigndocs"
        
        log.info(f"На URL = {req_url}")
        log.info(f"Відправляю запит: {req_body}")
        response = requests.post(req_url,  data=req_body , headers={"Content-Type": "application/json"} ) 
        log.info(f"Отримана відповідь: {response.text}")

        log.info("Перевія statusCode=200")
        self.assertEqual(response.status_code, 200)
        log.info("Перетворюю відповідь в JSON")
        res_dict=response.json()  
        log.info("Перевіряю наявність обов'язкових ключів у JSON")
        
        log.info("Ключ Header присутній")
        self.assertIn("Header", res_dict.keys())
        
        log.info("Ключ Header.Status присутній")
        self.assertIn("Status", res_dict["Header"].keys())
        
        log.info("Значення ключа Header.Status == OK")         
        self.assertEqual( res_dict["Header"]["Status"], "OK")         

        log.info("Ключ Response присутній")
        self.assertIn("Response", res_dict.keys())

        log.info("Перевіряю наявність ключ deeplink")
        self.assertIn("deepLink", res_dict["Response"].keys())

        log.info(f"Отриманий deep link відправте на мобільний пристрій deeplink: { res_dict["Response"]["deepLink"] }")




#unittest.main(argv=[''], verbosity=2, exit=False)  

def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestNotebook('test_sign_docs'))
    #suite.addTest(TestNotebook('test_dbrecord'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())


INFO:setUp:=======Підключаюся до бази даних===============
INFO:test_sign_docs:Опис тесту: ================ Пошук заявки на підпис документів через ДІЯ ================
INFO:test_sign_docs:Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ
INFO:test_sign_docs:Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)
INFO:test_sign_docs:Вибрано row: {'IDPROCESS': '5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3', 'ZOSYS_ID': 9380, 'FID': 4863276, 'DOCCNT': 1}
INFO:test_sign_docs:Формую json структуру для http запиту
INFO:test_sign_docs:На URL = http://prx-int-ace1:7816/apptoapp_srvc/v1/diiasigndocs
INFO:test_sign_docs:Відправляю запит: {"Header": {"SystemId": "XZ", "CallID": "XZ-111-5", "DtRequest": "2024-07-30T12:05:49.440Z"}, "Request": {"src": "SEBFL.ONBOARDING", "idprocess": "5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3", "returnLink": "https://domain/returnlink"}}
INFO:test_sign_docs:Отримана відповідь: {"H

.
----------------------------------------------------------------------
Ran 1 test in 0.932s

OK


**На цьому процес розривається**
**Після закінчення процесу підписання в ДІЯ виконується другий крок тестування: Отримання підписів на документи** 

## Отримаємо даниі про підпис з бази даних, так як як http методів отримання не передбачалося
В найпростішому випадку достатньо перевірити, що значення в  полях підпису та дати внесення підпису не **NULL** або в термінах PYthon  не **None**, тобто мають такі значення:
'ECP_0_DT': datetime.datetime(2024, 11, 2, 17, 51, 53)
'ECP_0': <oracledb.LOB object at 0x000001AF9AE9C530>

In [34]:
import requests
import  unittest
import sys
import logging
import oracledb
import pytest
import json
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

class TestNotebookSignGet(unittest.TestCase):
   
    
    def setUp(self):
        log=logging.getLogger('setUp')
        log.info("=======Підключаюся до бази даних===============")
        self.connection = oracledb.connect(dsn)

    def tearDown(self):
        log=logging.getLogger('tearDown')
        log.info("=======Відключаюся від бази даних ===============")
        self.connection.close()

  
    def test_getsigns(self):
        """================ Запит на отримання електронно-цифрових підписів на пакет документів ================"""
        log=logging.getLogger('test_sign_docs')
        log.info( f"Опис тесту: {self.shortDescription()}" )

        r = TestNotebook.shared_data
        log.info( f"  {TestNotebook.shared_data["src"] }")
        log.info( f" {TestNotebook.shared_data["idprocess"] } ")      

        log.info(f"Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ")

         #t.ecp_0 
        
        log.info(f"Вибираю з бази даних пакет документів та первіряю значння ecp_0  на null")
        with self.connection.cursor() as cursor:
            sql = """select  x.idprocess , z.zosys_id, t.fid, t.edoc_type, t.ecp_0, t.ecp_0_dt from qpf.QPF$FXCUST_ZF_EDOC t
                              inner join qpf.QPF$FXCUST_ZF z on z.fid=t.fid 
                              inner join qpf.QPF$FXCUST_ZF$EXTID x on x.zosys_id=z.zosys_id
                   where x.idprocess=:a_procid
                         AND t.edoc_type IN ('AML_OPROS','CASA_CABBAGE_CONTRCOMPLEX','NOTE_FGVFO','CARDACC_FORM', 'CRS_I', 'EDR_FORM', 'CASA_CABBAGE_FOPCOMPLEX', 'CURRACC_FORM')"""
            cursor.execute(sql, {"a_procid": TestNotebook.shared_data["idprocess"] })
            log.info("Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)")
            columns = [col[0] for col in cursor.description]
            cursor.rowfactory = lambda *args: dict(zip(columns, args))
            dbdata = cursor.fetchall()
            log.info(f"Вибрано row: {dbdata}")
            for row in dbdata:
                print( f"Отримано підпис: {row}  ")
                log.info("Поле з ЕЦП ECP_0 не повинно бути null")
                self.assertIsNotNone(row["ECP_0"])
                log.info("Поле з датою внесення ЕЦП ECP_0_DT  не повинно бути null")
                self.assertIsNotNone(row["ECP_0_DT"])
                
            TestNotebook.shared_data["signeddata"] = dbdata
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestNotebookSignGet('test_getsigns'))

    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())

INFO:setUp:=======Підключаюся до бази даних===============
INFO:test_sign_docs:Опис тесту: ================ Запит на отримання електронно-цифрових підписів на пакет документів ================
INFO:test_sign_docs:  SEBFL.ONBOARDING
INFO:test_sign_docs: 5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3 
INFO:test_sign_docs:Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ
INFO:test_sign_docs:Вибираю з бази даних пакет документів та первіряю значння ecp_0  на null
INFO:test_sign_docs:Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)
INFO:test_sign_docs:Вибрано row: [{'IDPROCESS': '5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3', 'ZOSYS_ID': 9380, 'FID': 4863276, 'EDOC_TYPE': 'AML_OPROS', 'ECP_0': <oracledb.LOB object at 0x000001AF9AE9DB80>, 'ECP_0_DT': datetime.datetime(2024, 11, 2, 17, 51, 53)}]
Отримано підпис: {'IDPROCESS': '5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3', 'ZOSYS_ID': 93

.
----------------------------------------------------------------------
Ran 1 test in 0.288s

OK


## Пробуєм читать ORACLE.BLOBS і ORACLE.DATA  
**Вибираємо з бази даних ще раз  ЕЦП та контент файлу як ORACLE.BLOB і виконуємо перевірку ЕЦП**
**Ну, за одно відформатеємо отриману дату типу ORACLE.DATA**

In [68]:
import requests
import  unittest
import sys
import logging
import oracledb
import pytest
import json
import datetime
import base64
from requests_toolbelt.multipart.encoder import MultipartEncoder
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

class TestNotebookBlobs(unittest.TestCase):
   
    
    def setUp(self):
        log=logging.getLogger('setUp')
        log.info("=======Підключаюся до бази даних===============")
        self.connection = oracledb.connect(dsn)

    def tearDown(self):
        log=logging.getLogger('tearDown')
        log.info("=======Відключаюся від бази даних ===============")
        self.connection.close()

  
    def test_blobs(self):
        """================ Пробуємо читати ORACLE BLOBS і виконати  перевірку ЕЦП================"""
        log=logging.getLogger('test_blobs')
        log.info( f"Опис тесту: {self.shortDescription()}" )

        r = TestNotebook.shared_data
        log.info( f"  {TestNotebook.shared_data["src"] }")
        log.info( f" {TestNotebook.shared_data["idprocess"] } ")      

        log.info(f"Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ")
        
  
        with self.connection.cursor() as cursor:
            sql = """select  x.idprocess , z.zosys_id, t.fid, t.edoc_type, t.edoc_hesh,  t.edoc_cont,  t.ecp_0, t.ecp_0_dt from qpf.QPF$FXCUST_ZF_EDOC t
                              inner join qpf.QPF$FXCUST_ZF z on z.fid=t.fid 
                              inner join qpf.QPF$FXCUST_ZF$EXTID x on x.zosys_id=z.zosys_id
                   where x.idprocess=:a_procid
                         AND t.edoc_type IN ('AML_OPROS','CASA_CABBAGE_CONTRCOMPLEX','NOTE_FGVFO','CARDACC_FORM', 'CRS_I', 'EDR_FORM', 'CASA_CABBAGE_FOPCOMPLEX', 'CURRACC_FORM')"""
            cursor.execute(sql, {"a_procid": TestNotebook.shared_data["idprocess"] })
            log.info("Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)")
            columns = [col[0] for col in cursor.description]
            cursor.rowfactory = lambda *args: dict(zip(columns, args))
            dbdata = cursor.fetchall()
            for row in dbdata:
                log.info(f' Форматую отраму ORACLE.DATA  Дата-час підпису { row["ECP_0_DT"].strftime("%Y-%m-%d %H:%M:%S")}')
                log.info( f'Виводжу тип докуаенту EDOC_TYPE =  {row["EDOC_TYPE"]}')

                log.info("Читаю отриманий ORACLE.BLOB ECP_0  (ЕЦП) та перевожу його в python string")
                ecp_orablob=row["ECP_0"]
                ecp_pyb=ecp_orablob.read()
                ecp_pys=ecp_pyb.decode("utf-8")

                log.info("Читаю отриманий ORACLE.BLOB EDOC_CONT (Контент файлу) та перевожу його в python string")
                cont_orablob=row["EDOC_CONT"]
                cont_pyb=cont_orablob.read()
                cont_pys=cont_pyb.decode("utf-8")

                log.info("Формую http запит  для перевірки ЕЦП криптографічним сервісом ДІЯ ")    
                req_url="http://pvx-diy-sign-pvx-diy-int.apps.okd4-cl-03.openshift.pravex.ua/api/chkcustsign"
                mp_encoder = MultipartEncoder(
                    fields={
                        "FileName": "FileData", 
                        "FileData": cont_pys,
                        "SignFileName": row["EDOC_TYPE"], 
                        "SignFileData": ecp_pys
                    }   
                )
                log.info("Виконю http запит")
                response = requests.post(req_url,  data=mp_encoder , headers={'Content-Type': mp_encoder.content_type} )    
                #log.info(f"Response: {response.text}")
                log.info("Перевіря відповідь  statusCode=200")
                self.assertEqual(response.status_code, 200)
                res_dict=response.json()
                log.info("Перевіряю наявність ключ ok")
                self.assertIn("ok", res_dict.keys())
        
                log.info("Значення ключа ok=True")
                self.assertEqual( res_dict["ok"], True)    


                log.info("Перевіряю наявність ключ signinfo")
                self.assertIn("signinfo", res_dict.keys())
                
                res_dict_S=json.dumps(  res_dict , ensure_ascii=False )
                
                log.info(f"Отримана відповідь: { res_dict_S }")

            
                
            TestNotebook.shared_data["signeddata"] = dbdata
def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestNotebookBlobs('test_blobs'))

    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())

INFO:setUp:=======Підключаюся до бази даних===============
INFO:test_blobs:Опис тесту: ================ Пробуємо читати ORACLE BLOBS і виконати  перевірку ЕЦП================
INFO:test_blobs:  SEBFL.ONBOARDING
INFO:test_blobs: 5B46A1AA64AFDFB4C3AED87817CA6CF2967B6B4CDC15B47357151AD7A72194E3 
INFO:test_blobs:Вибираю з бази даних пакет документів OnBoarding для підпису через ДІЯ
INFO:test_blobs:Вибираю idporcess заявки на підпис пакету документів (перший вибраний запис)
INFO:test_blobs: Форматую отраму ORACLE.DATA  Дата-час підпису 2024-11-02 17:51:53
INFO:test_blobs:Виводжу тип докуаенту EDOC_TYPE =  AML_OPROS
INFO:test_blobs:Читаю отриманий ORACLE.BLOB ECP_0  (ЕЦП) та перевожу його в python string
INFO:test_blobs:Читаю отриманий ORACLE.BLOB EDOC_CONT (Контент файлу) та перевожу його в python string
INFO:test_blobs:Формую http запит  для перевірки ЕЦП криптографічним сервісом ДІЯ 
INFO:test_blobs:Виконю http запит
INFO:test_blobs:Перевіря відповідь  statusCode=200
INFO:test_blobs:Переві

.
----------------------------------------------------------------------
Ran 1 test in 0.718s

OK
